### ランダムフォレスト　パラメータチューニング

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(55323, 12)

In [4]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [5]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

### 検証するパラメータの設定

In [ ]:
grid_param = {
    'n_estimators': [60, 80, 100, 120],
    'criterion': ['mse'],
    'max_depth': [i for i in range(1, 11, 2)],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [6]:
grid_param = {
    'n_estimators': [60, 70, 80, 90, 100],
    'criterion': ['mse'],
    'max_depth': [3, 5, 6, 7, 9],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [ ]:
def rmse(y, y_pred):
    ret = np.sqrt(mean_squared_error(y, y_pred))
    print("RMSE=", ret)
    return ret

### グリッドサーチ

In [7]:
gs = GridSearchCV(estimator=RandomForestRegressor(**grid_param), param_grid=grid_param, scoring=make_scorer(mean_squared_error, greater_is_better=False), cv=5, return_train_score=False)

In [8]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion=['mse'],
                                             max_depth=[3, 5, 6, 7, 9],
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=[1, 3, 5, 7, 9],
                                             min_samples_split=[2, 4, 6, 8, 10],
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=[60, 70...
                                             warm_start=False),
             iid='deprecated', n_jobs=None,
   

### 一番良かったパラメータで学習

In [9]:
gs.best_score_

-16791.134603744584

In [10]:
gs.best_params_

{'criterion': 'mse',
 'max_depth': 9,
 'min_samples_leaf': 7,
 'min_samples_split': 2,
 'n_estimators': 100,
 'random_state': 42}

In [11]:
RF = RandomForestRegressor(**gs.best_params_)

In [12]:
RF = RF.fit(train_X, train_y)

In [13]:
RF.feature_importances_

array([0.12797395, 0.22413023, 0.46171355, 0.02659484, 0.07288789,
       0.00442329, 0.00202128, 0.0022573 , 0.00072803, 0.07726963])

In [14]:
sorted(
    zip(map(lambda x: round(x, 3), RF.feature_importances_), train.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11]].columns),
    reverse=True)

[(0.462, 'bedrooms'),
 (0.224, 'bathrooms'),
 (0.128, 'accommodates'),
 (0.077, 'city_flag'),
 (0.073, 'host_response_rate'),
 (0.027, 'beds'),
 (0.004, 'number_of_reviews'),
 (0.002, 'property_type_num'),
 (0.002, 'bed_type_flag'),
 (0.001, 'cancel_policy_flag')]

In [15]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(RF, out_file=None,
#                                feature_names=train.iloc[:, 1:].columns,
#                                class_names=train.Survived.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

### テストデータで予測

In [16]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [17]:
test_x.shape, test_y.shape

((11065, 10), (11065,))

In [18]:
pred_y = RF.predict(test_x)

In [19]:
np.sqrt(mean_squared_error(test_y, pred_y))

131.06166229358334

### 検証データで予測

In [ ]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [ ]:
valid.shape

In [ ]:
# ID の保存
valid_pass = valid.id.values

In [ ]:
valid_X = valid.iloc[:, 1:].values

In [ ]:
# valid_X.describe()

In [ ]:
valid_X.shape, train_X.shape

In [ ]:
pred_valid_y = RF.predict(valid_X)

In [ ]:
pred_valid_y.shape

In [ ]:
type(valid_pass), type(pred_valid_y)

In [ ]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [ ]:
result_df.to_csv("./RF_5.csv", header=False)